In [ ]:
"base version"
from pymongo import MongoClient

database_name = "OrgqrS1HZ"
collection_name = "userStories"
milvus_collection_name = "temp" 
mongodb_uri = "mongodb://localhost:27018"

mongo_client = MongoClient(mongodb_uri)
db = mongo_client[database_name]
collection = db[collection_name]
documents = collection.find()
for document in documents:
    print(document)

In [ ]:
"final user story version"

In [ ]:
import pandas as pd
from setfit import SetFitModel, Trainer
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Load data from CSV
data = pd.read_csv("RAG Evaluation(classifier).csv", encoding="ISO-8859-1")  # Replace with your CSV file name

# Extract queries and labels
queries = data["query"].tolist()
labels = data["binary"].apply(lambda x: 1 if x == "TRUE" else 0).tolist()  # Convert 'True'/'False' to 1 and 0

# Split data into training and testing sets
train_queries, test_queries, train_labels, test_labels = train_test_split(queries, labels, test_size=0.2, random_state=42)

# Load a pre-trained sentence transformer model for SetFit
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

# Train the SetFit model on the few-shot examples
trainer = SetFitTrainer(
    model=model,
    train_dataset=list(zip(train_queries, train_labels)),
    eval_dataset=list(zip(test_queries, test_labels)),
    loss_class="CosineSimilarityLoss",
    metric="accuracy"
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate()
print("Accuracy:", metrics["accuracy"])

# Define a function to classify a new user query
def classify_query(query):
    prediction = model.predict([query])
    label = "True" if prediction[0] == 1 else "False"
    return label

# Test the classifier on a new user query
user_query = "Provide me a software development user story"
label = classify_query(user_query)
print(f"The query is classified as: {label}")

In [ ]:
import pandas as pd
from setfit import SetFitModel, Trainer
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Load data from CSV
data = pd.read_csv("RAG Evaluation(classifier).csv", encoding="ISO-8859-1")  # Replace with your CSV file name

# Extract queries and labels
queries = data["query"].tolist()
labels = data["binary"].apply(lambda x: 1 if x == "TRUE" else 0).tolist()  # Convert 'True'/'False' to 1 and 0

# Split data into training and testing sets
train_queries, test_queries, train_labels, test_labels = train_test_split(queries, labels, test_size=0.2, random_state=42)

# Convert training and testing sets to Dataset objects
train_dataset = Dataset.from_dict({"text": train_queries, "label": train_labels})
test_dataset = Dataset.from_dict({"text": test_queries, "label": test_labels})

# Load a pre-trained sentence transformer model for SetFit
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

# Use Trainer instead of SetFitTrainer
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    metric="accuracy",
    samples_per_label=110,  # Adjust based on data size and balance
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate()
print("Accuracy:", metrics["accuracy"])

# Define a function to classify a new user query
def classify_query(query):
    prediction = model.predict([query])
    label = "True" if prediction[0] == 1 else "False"
    return label

# Test the classifier on a new user query
user_query = "Provide me a software development user story"
label = classify_query(user_query)
print(f"The query is classified as: {label}")


In [ ]:
pip install flair

In [ ]:
# Install necessary packages if not already installed
# !pip install flair pandas

import pandas as pd
from flair.data import Sentence
from flair.models import TARSClassifier
from sklearn.model_selection import train_test_split

# Load data from CSV
data = pd.read_csv("RAG Evaluation(classifier).csv", encoding="ISO-8859-1")

# Convert labels to binary (1 for "TRUE", 0 for "FALSE")
data["binary"] = data["binary"].apply(lambda x: 1 if x == "TRUE" else 0)

# Split data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42, stratify=data["binary"])

# Initialize the TARS model for text classification
tars = TARSClassifier.load("tars-base")
tars.add_and_switch_to_new_task("Tech Query Classification", label_dictionary=["TRUE", "FALSE"])

# Training the TARS model on the few-shot examples
for index, row in train_data.iterrows():
    sentence = Sentence(row["query"])
    sentence.set_label("Tech Query Classification", "TRUE" if row["binary"] == 1 else "FALSE")
    tars.predict(sentence)

# Evaluate model on the test dataset
correct, total = 0, len(test_data)
for index, row in test_data.iterrows():
    sentence = Sentence(row["query"])
    tars.predict(sentence)
    predicted_label = sentence.labels[0].value
    correct += int(predicted_label == ("TRUE" if row["binary"] == 1 else "FALSE"))

print("Accuracy on test data:", correct / total)

# Define a function to classify a new user query
def classify_query(query):
    sentence = Sentence(query)
    tars.predict(sentence)
    return sentence.labels[0].value

# Test the classifier on a new user query
user_query = "Provide me a software development user story"
print("The query is classified as:", classify_query(user_query))


In [ ]:
from tqdm import tqdm
from tars import TARS  # Corrected import
from tars.data import Corpus
from tars.trainer import ModelTrainer
from datasets import Dataset

# Load your data
import pandas as pd

# Load data from CSV
data = pd.read_csv("RAG Evaluation(classifier).csv", encoding="ISO-8859-1")  # Replace with your CSV file name

# Extract queries and labels
queries = data["query"].tolist()
labels = data["binary"].apply(lambda x: 1 if x == "TRUE" else 0).tolist()  # Convert 'True'/'False' to 1 and 0

# Create a Dataset using HuggingFace Dataset
dataset_dict = {
    "train": {"query": queries[:int(0.8 * len(queries))], "label": labels[:int(0.8 * len(labels))]},
    "test": {"query": queries[int(0.8 * len(queries)):], "label": labels[int(0.8 * len(labels)):]}
}

# Create Dataset object for TARS
train_dataset = Dataset.from_dict(dataset_dict["train"])
test_dataset = Dataset.from_dict(dataset_dict["test"])

# Create the corpus
corpus = Corpus(train=train_dataset, test=test_dataset)

# Initialize the TARS model
tars = TARS.load('tars-base')  # Updated to correctly load TARS model

# Define the label dictionary and task name
label_dict = {"TRUE": 0, "FALSE": 1}

# Add a new label type and switch to the new task
tars.add_and_switch_to_new_task(label_type="tech_classifier", label_dictionary=label_dict, task_name="tech_classification")

# Define the trainer
trainer = ModelTrainer(tars, corpus)

# Train the model with progress bar
tqdm.write("Training the model...")
trainer.train(
    base_path="tars_model",
    learning_rate=3e-5,
    mini_batch_size=8,
    max_epochs=3
)

# Evaluate the model after training
results = trainer.evaluate_model(corpus.test)
print("Evaluation Results:", results)

# Function to classify new queries
def classify_query(query):
    prediction = tars.predict([query])
    label = "TRUE" if prediction[0] == 0 else "FALSE"
    return label

# Test the classifier on a new user query
user_query = "Provide me a software development user story"
label = classify_query(user_query)
print(f"The query is classified as: {label}")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from tqdm import tqdm

# Load and preprocess your CSV data
data = pd.read_csv("RAG Evaluation(classifier).csv", encoding="ISO-8859-1")  # Path to your CSV

# Convert 'TRUE'/'FALSE' to 1/0
data['label'] = data['binary'].apply(lambda x: 1 if x == "TRUE" else 0)

# Split data into train and test sets (80% train, 20% test)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data[['query', 'label']])
test_dataset = Dataset.from_pandas(test_data[['query', 'label']])

# Load tokenizer and model
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenize datasets with padding
def tokenize_function(examples):
    return tokenizer(examples["query"], padding="max_length", truncation=True, max_length=64)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Determine device (MPS or CPU)
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

# Move model to selected device
model.to(device)

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs'
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Training with progress bar
print("Starting training...")
for _ in tqdm(range(int(training_args.num_train_epochs))):
    trainer.train()

# Evaluation
metrics = trainer.evaluate()
print("Evaluation Results:", metrics)

# Classification function for new queries
def classify_query(query):
    inputs = tokenizer(query, return_tensors="pt", padding="max_length", truncation=True, max_length=64)
    inputs = inputs.to(device)  # Move inputs to the selected device
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    return "TRUE" if predicted_class_id == 1 else "FALSE"

# Test the classifier
user_query = "Provide me a software development user story"
label = classify_query(user_query)
print(f"The query is classified as: {label}")

eval_metrics = trainer.evaluate()
print(f"Accuracy: {eval_metrics['eval_accuracy']}")

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch

# Step 1: Load your CSV data
# Replace 'your_file.csv' with the actual path to your CSV
data = pd.read_csv("RAG Evaluation(classifier).csv", encoding="ISO-8859-1")  # Replace with your CSV file name

# Step 2: Prepare your dataset
# Assuming your CSV has 'query' as the input text and 'binary' as the label (TRUE/FALSE)
data["label"] = data["binary"].apply(lambda x: 1 if x == "TRUE" else 0)  # Convert 'TRUE'/'FALSE' to 1/0

# Convert to Hugging Face Dataset format
dataset = Dataset.from_pandas(data[['query', 'label']])

# Step 3: Tokenizer
model_name = "facebook/bart-large-mnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples["query"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Step 4: Load the pre-trained model with the correct number of labels
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)

# Step 5: Set up the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    weight_decay=0.01,
    load_best_model_at_end=True,  # Load the best model based on evaluation results
)

# Step 6: Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,  # Using the same data for evaluation
    compute_metrics=lambda p: {'accuracy': (p.predictions.argmax(axis=-1) == p.label_ids).mean()},
)

# Step 7: Train the model
trainer.train()

# Step 8: Evaluate the model
eval_results = trainer.evaluate()

# Print the evaluation results (including accuracy)
print("Evaluation Results:", eval_results)

# Step 9: Inference (Classifying New Queries)
def classify_query(query):
    inputs = tokenizer(query, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=1).item()
    return "TRUE" if prediction == 1 else "FALSE"

# Example: Test with a new query
query = "Provide a software development user story"
print(f"The query is classified as: {classify_query(query)}")


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import pipeline

# Load your own CSV file with queries and binary labels
data = pd.read_csv("RAG Evaluation(classifier).csv", encoding="ISO-8859-1")  # Replace with your CSV file

# Assuming your CSV contains columns 'query' and 'binary' where 'binary' is either "TRUE" or "FALSE"
queries = data["query"].tolist()
labels = data["binary"].apply(lambda x: "TRUE" if x == "TRUE" else "FALSE").tolist()

# You can split your data into training and testing sets (you can also change the test_size as per your need)
train_queries, test_queries, train_labels, test_labels = train_test_split(queries, labels, test_size=0.2, random_state=42)

# Initialize the zero-shot classification pipeline with a pre-trained model
zero_shot_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Define the labels you're interested in
candidate_labels = ["TRUE", "FALSE"]

# Run zero-shot classification on the test set
predictions = []
for query in test_queries:
    result = zero_shot_classifier(query, candidate_labels)
    predictions.append(result["labels"][0])  # Get the top label as the prediction

# Evaluate the model
print(classification_report(test_labels, predictions))


In [ ]:
print(data["binary"].value_counts())

In [ ]:
from transformers import Trainer, TrainingArguments, BertForSequenceClassification, BertTokenizer
from datasets import Dataset

# Convert data to HuggingFace format
data_dict = {
    "text": train_queries,
    "label": train_labels
}
train_dataset = Dataset.from_dict(data_dict)

data_dict = {
    "text": test_queries,
    "label": test_labels
}
test_dataset = Dataset.from_dict(data_dict)

# Load a pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Tokenize the datasets
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set up the Trainer
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="epoch",     # evaluate every epoch
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    num_train_epochs=3,              # number of training epochs
)

trainer = Trainer(
    model=model,                         # the model
    args=training_args,                  # training arguments
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import pipeline
from sklearn.metrics import classification_report

# Step 1: Load your CSV file
data = pd.read_csv("RAG Evaluation(classifier).csv", encoding="ISO-8859-1")

# Step 2: Inspect the first few rows of your data (optional)
print(data.head())

# Step 3: Clean up the 'binary' column: map 'TRUE' to 1 and 'FALSE' to 0
data['binary'] = data['binary'].apply(lambda x: 1 if x == "TRUE" else 0)

# Step 4: Split the dataset into training and testing sets (80% training, 20% testing)
train_queries, test_queries, train_labels, test_labels = train_test_split(
    data['query'], data['binary'], test_size=0.2, random_state=42
)

# Step 5: Load the zero-shot classification pipeline from Hugging Face
zero_shot_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Step 6: Define the candidate labels for classification (these are your classes)
candidate_labels = ["TRUE", "FALSE"]

# Step 7: Function to classify a query using the zero-shot classifier
def classify_query(query):
    result = zero_shot_classifier(query, candidate_labels)
    return result['labels'][0], result['scores'][0]

# Step 8: Classify the queries in the test set and collect the predictions
predictions = [classify_query(query)[0] for query in test_queries]

# Step 9: Print the first 10 predictions (optional)
print(predictions[:10])

# Step 10: Evaluate the model using scikit-learn's classification_report
print("Evaluation Results:")
print(classification_report(test_labels, predictions))



In [ ]:
pip install setfit sentence-transformers

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from setfit import SetFitModel, Trainer
from sentence_transformers import SentenceTransformer
from sklearn.metrics import classification_report

# Step 1: Load your dataset (replace 'your_data.csv' with the actual path to your CSV)
data = pd.read_csv('your_data.csv')

# Step 2: Ensure the 'binary' column has 0 (FALSE) and 1 (TRUE) values
data['binary'] = data['binary'].apply(lambda x: 1 if x == 'TRUE' else 0)

# Step 3: Split your dataset into train and test (using stratified split for balanced classes)
train_queries, test_queries, train_labels, test_labels = train_test_split(
    data['query'], data['binary'], test_size=0.2, stratify=data['binary']
)

# Step 4: Load the pre-trained sentence transformer model
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

# Step 5: Initialize the SetFit model with a pre-trained transformer model
model = SetFitModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Step 6: Prepare the training dataset (list of tuples of query and label)
train_dataset = list(zip(train_queries, train_labels))
test_dataset = list(zip(test_queries, test_labels))

# Step 7: Define and train the model using the Trainer class
trainer = Trainer(model=model, train_dataset=train_dataset, eval_dataset=test_dataset)

# Train the model (this will use few-shot learning for the task)
trainer.train()

# Step 8: Evaluate the model on the test set
y_pred = trainer.predict(test_queries)

# Step 9: Output the classification report (accuracy, precision, recall, etc.)
print(classification_report(test_labels, y_pred))

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv('RAG Evaluation(classifier).csv', encoding="ISO-8859-1")
X = df['query']
y = df['binary']
vectorizer = TfidfVectorizer(max_features=420)
X_embeddings = vectorizer.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_embeddings, y, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

def predict_query(query_text, model, vectorizer):
    query_embedding = vectorizer.transform([query_text])

    prediction = model.predict(query_embedding)
    
    return "True" if prediction[0] == 1 else "False"

# Example
new_query = "Give me all the stories?"
print("Prediction for the input query:", predict_query(new_query, model, vectorizer))

Accuracy: 0.8571428571428571
              precision    recall  f1-score   support

       False       1.00      0.73      0.85        45
        True       0.76      1.00      0.87        39

    accuracy                           0.86        84
   macro avg       0.88      0.87      0.86        84
weighted avg       0.89      0.86      0.86        84

Prediction for the input query: True


Prediction for the input query: True


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

# Load and prepare data
df = pd.read_csv('RAG Evaluation(classifier).csv', encoding="ISO-8859-1")
X = df['query']
y = df['binary']
vectorizer = TfidfVectorizer(max_features=420)
X_embeddings = vectorizer.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_embeddings, y, test_size=0.2, random_state=42)

# Define function to train and evaluate model
def evaluate_model(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"\nModel: {model.__class__.__name__}")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))

# Initialize and evaluate models
models = [
    LogisticRegression(),
    DecisionTreeClassifier(),
    SVC(),
    MultinomialNB()
]

for model in models:
    evaluate_model(model)



Model: LogisticRegression
Accuracy: 0.8571428571428571
              precision    recall  f1-score   support

       False       1.00      0.73      0.85        45
        True       0.76      1.00      0.87        39

    accuracy                           0.86        84
   macro avg       0.88      0.87      0.86        84
weighted avg       0.89      0.86      0.86        84


Model: DecisionTreeClassifier
Accuracy: 0.8571428571428571
              precision    recall  f1-score   support

       False       0.95      0.78      0.85        45
        True       0.79      0.95      0.86        39

    accuracy                           0.86        84
   macro avg       0.87      0.86      0.86        84
weighted avg       0.87      0.86      0.86        84


Model: SVC
Accuracy: 0.9523809523809523
              precision    recall  f1-score   support

       False       1.00      0.91      0.95        45
        True       0.91      1.00      0.95        39

    accuracy             

In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

df = pd.read_csv('RAG Evaluation(classifier).csv', encoding="ISO-8859-1")
X = df['query']
y = df['binary']
vectorizer = TfidfVectorizer(max_features=420)
X_embeddings = vectorizer.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_embeddings, y, test_size=0.4, random_state=42)
model = SVC()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

def predict_query(query_text, model, vectorizer):
    query_embedding = vectorizer.transform([query_text])

    prediction = model.predict(query_embedding)
    
    return "True" if prediction[0] == 1 else "False"

# Example
new_query = "Give me all the stories?"
print("Prediction for the input query:", predict_query(new_query, model, vectorizer))

Accuracy: 0.8809523809523809
              precision    recall  f1-score   support

       False       0.97      0.76      0.85        76
        True       0.83      0.98      0.90        92

    accuracy                           0.88       168
   macro avg       0.90      0.87      0.88       168
weighted avg       0.89      0.88      0.88       168

Prediction for the input query: True


In [37]:
new_query = ""
print("Prediction for the input query:", predict_query(new_query, model, vectorizer))

Prediction for the input query: False
